In [23]:
from abc import ABC, abstractmethod
from typing import Optional, Dict, List, Tuple, Any
import logging
from setup.logging_config import setup_logging
import psycopg2 as pg
import clickhouse_connect
import os
import yaml
from pathlib import Path

setup_logging()

class DQParser:
    __version: str = 'v0.0.1'
    
    def __init__(self, config_path: str):
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info(f'Version {self.version}')
        
        self.config_path = Path(config_path).resolve()
        
        if self.config_path.is_file() and self.config_path.suffix in ('.yaml', '.yml'):
            self._parse_config(self.config_path)
        elif self.config_path.is_dir():
            for config_file in self.config_path.glob('*.y*ml'):
                self._parse_config(config_file)
        else:
            self.logger.warning(f'Provided path {self.config_path} is invalid has no YAML/YML files.')
 
    def _parse_config(self, config_path):
        try:
            with config_path.open('r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
        except Exception as e:
            self.logger.error(f'Failed to load config {config_path}: {e}')
        
        print(config)
    
    @property
    def version(self) -> str:
        return self.__version
    

In [24]:
obj = DQParser('/home/kaboupi/repos/dq-app/dq_configs')

2025-04-23 11:47:07 | DQParser | INFO | Version v0.0.1


{'kind': 'apps/DQ', 'version': 'v1', 'metadata': {'project': 'customers_analytics', 'name': 'sales_data', 'annotations': {'owner': 'Georgiy Novozhilov', 'description': 'Sample DQ script for table `sales_data` both in:\n  - Postgres\n  - Clickhouse'}}, 'tasks': {'transfer_postgres_weekly': {'kind': 'TransferSQL', 'version': 'v1', 'spec': {'asset': {'domain': 'misis', 'key': 'facb8e28-3006-4414-9737-a4e4bd2db62b', 'description': 'Weekly turnover for Postgres', 'connection': 'postgres_db_conn', 'type': 'postgres'}, 'params': {'type': 'SQL', 'query': 'select\n  date_trunc(\'week\', "Date") :: DATE valid_from_dttm\n  , SUM("Cost") as cost\n  , SUM("Revenue") as revenue\nfrom misis_project.sales_data sd  group by 1 order by 1;', 'fields': {'dimDate': 'valid_from_dttm'}}}}, 'transfer_clickhouse_weekly': {'kind': 'TransferSQL', 'version': 'v1', 'spec': {'asset': {'domain': 'misis', 'key': 'facb8e28-3006-4414-9737-a4e4bd2db62b', 'description': 'Weekly turnover for Clickhouse', 'connection': 'cl